# Contents:
- [1. Import Libraries & Data](#import-libraries)
- [2. Data Preprocessing](#data-preprocessing)
- [3. Models Experiments](#models)
    - [3.1 All Features](#All-Features)
    - [3.2 Feature-Selection](#Feature-Selection)
    - [3.3 Feature Selection using PCA](#Feature-Selection-PCA)
- [4. ANN](#ann)

<a id="import-libraries"></a>
# 1. Import Libraries & Data

In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import tensorflow as tf
import pickle
import random
SEED = 1
np.random.seed(SEED)

#warnings.filterwarnings('ignore')

# preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# models 
from sklearn.linear_model import LinearRegression
from sklearn. linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import BayesianRidge
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from lightgbm import LGBMRegressor
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline

# evaluation
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [89]:
data = pd.read_csv('../../data/final/clean_trial4.csv')

In [90]:
data.head(2)

,logkpl,Compound,SMILES,Texpi,ALogp2,nAcid,nAromBond,ATSc2,ATSc3,ATSc4,...,nRings9,TopoPSA,VAdjMat,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,XLogP,Zagreb
0,-3.55,Urea,C(=O)(N)N,312,1.085972,0,0,-0.116019,0.023614,0.0,...,0,69.11,2.584963,1.683013,6.732051,2.244017,4.488034,9.0,-1.686,12.0
1,-3.69,Urea,C(=O)(N)N,312,1.085972,0,0,-0.116019,0.023614,0.0,...,0,69.11,2.584963,1.683013,6.732051,2.244017,4.488034,9.0,-1.686,12.0


In [91]:
smiles_df = data[['SMILES', 'Compound']]
smiles_df.head(2)

,SMILES,Compound
0,C(=O)(N)N,Urea
1,C(=O)(N)N,Urea


In [92]:
data.shape

(417, 149)

In [93]:
data[data.duplicated()]

,logkpl,Compound,SMILES,Texpi,ALogp2,nAcid,nAromBond,ATSc2,ATSc3,ATSc4,...,nRings9,TopoPSA,VAdjMat,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,XLogP,Zagreb


In [94]:
# no missing values
data.isna().sum().sum()

0

<a id="data-preprocessing"></a>
# 2. Data Preprocessing

In [95]:
model_data = data.drop(["SMILES", 'Compound'], axis=1)

In [96]:
X = model_data.drop(["logkpl"], axis=1)
y = model_data['logkpl']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.15, random_state=SEED)

print("Shape of X_train: {} \t Shape of y_train: {}".format(X_train.shape, y_train.shape))
print("Shape of X_test: {} \t Shape of y_test: {}".format(X_test.shape, y_test.shape))

Shape of X_train: (354, 146) 	 Shape of y_train: (354,)
Shape of X_test: (63, 146) 	 Shape of y_test: (63,)


In [36]:
with open('../../models/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

<a id="models"></a>
# 3. Models Experiments

In [97]:
def plot_feature_importance(model, cols, model_name, slice=20):
    importances = model.feature_importances_
    feature_names = cols#X.columns#selected_features_X.columns #

    # Create a pandas DataFrame with the feature importances
    df = pd.DataFrame({"feature": feature_names, "importance": importances})

    # Sort the DataFrame by importance score
    df = df.sort_values("importance", ascending=False).reset_index(drop=True)
    # df[:slice].to_excel("../../results/Feature Importance/{}_feature_importance.xlsx".format(model_name), index=False)

    # Create a bar plot using Seaborn
    plt.figure(figsize=(12, 8))
    sns.set_style("whitegrid")
    sns.barplot(x="importance", y="feature", data=df[:slice])
    plt.title("Top 20 Feature Importances {}".format(model_name))
    plt.ylabel("Feature Name")
    plt.xlabel("Importance")
    plt.show()

<a id="All-Features"></a>
### 3.1 All Features

In [98]:
def ANN_model(X_train, X_test, y_train, y_test):
    
    model = tf.keras.Sequential([

    tf.keras.layers.Dense(256, input_shape=[X_train.shape[1]]),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(8),
    tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0001), loss="mean_absolute_error")

    history = model.fit(X_train, y_train, epochs=2500, validation_data=(X_test, y_test), verbose=2)

    plt.title('Learning Curves')
    plt.xlabel('Epoch')
    plt.ylabel('MAE')
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.legend()
    plt.show()

    predictions = model.predict(X_test)

    # MAE, MSE, RMSE
    print("MAE: {}".format(mean_absolute_error(y_test, predictions)))
    print("MSE: {}".format(mean_squared_error(y_test, predictions)))
    print("RMSE: {}".format(mean_squared_error(y_test, predictions, squared=False)))
    print("MAPE: {}".format(mean_absolute_percentage_error(y_test, predictions)))
    print("R2: {}".format(r2_score(y_test, predictions)))

    final_result = pd.DataFrame({"predicted": model.predict(X_test).reshape(X_test.shape[0],), 
              "actual": y_test})
    
    # saving the model

# serialize model to JSON
    model_json = model.to_json()
    # with open("../../models/ANN_model.json", "w") as json_file:
    #     json_file.write(model_json)
    # # serialize weights to HDF5
    # model.save_weights("../../models/ANN_model.h5")
    print("Saved model to disk")

    # load json and create model
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# loaded_model.predict(X_test[0].reshape((1, 512, 512, 1))).argmax(axis=1)
# print("Loaded model from disk")
    
    return final_result.reset_index(drop=True)

In [99]:
def evaluate_model(model_df, i, model_name, model, X_train, y_train, X_test, y_test):
    """
    this function is for regression takes the model with the data and calculate
    the scores, with cross validation techniques, in addition to MAE, MSE, RMSE, MAPE
    R Squared and Adjusted R Squared

    :param model: model
    :param X_train, X_test, y_train, y_test: data that was used
    """

    # cross validation with 5 folds
    all_cv_5 = cross_val_score(model, X_train, y_train, cv=5, scoring="neg_mean_absolute_error")
    #print("all CV 5: {}".format(all_cv_5))
    # print("Mean Cross-Validation score: {}".format(all_cv_5.mean()))

    # predictions from our model
    predictions = model.predict(X_test)


    # calculating R squared and Adjusted R squared
    r_sqre = r2_score(y_test, predictions)
    n = len(y_test)
    p = X_test.shape[1] # number of independant features

    Adj_r2 = 1 - ((1 - r_sqre) * (n - 1) / (n - 1 - p))
    
    test_mae = mean_absolute_error(y_test, predictions)

    test_mse = mean_squared_error(y_test, predictions)
    test_rmse = np.sqrt(mean_squared_error(y_test, predictions))

    
    # Plotting
    plt.figure(figsize=(12, 8))
    plt.scatter(y_test, predictions, color='blue', label='Predicted vs Actual')
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--', label='Perfect Prediction')
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.title('Predicted vs Actual {}'.format(model_name))
    plt.legend()
    plt.show()
    #print("=" * 40)
    model_df.loc[i] = [model_name, all_cv_5.mean(),
                    test_mae, mean_absolute_percentage_error(y_test, predictions),
                   test_mse, test_rmse, r_sqre, Adj_r2]

    return model_df

In [100]:
def train_and_evalute(X_train, X_test, y_train, y_test, metric):
    np.random.seed(SEED)
    # scaler = StandardScaler()
    # X_scaled = scaler.fit_transform(X)

    # X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=42)

    # print("Shape of X_train: {} \t Shape of y_train: {}".format(X_train.shape, y_train.shape))
    # print("Shape of X_test: {} \t Shape of y_test: {}".format(X_test.shape, y_test.shape))

    # Building pipelins of standard scaler and model for varios regressors.

    lr = LinearRegression()

    lasso = Lasso()

    dt = DecisionTreeRegressor()

    rf = RandomForestRegressor()

    xgb = XGBRegressor()

    gbr = GradientBoostingRegressor()

    eln = ElasticNet()

    br = BayesianRidge()

    cat = CatBoostRegressor(allow_writing_files=False, verbose=0, task_type="GPU")

    lgbm = LGBMRegressor()



    # List of all the pipelines
    pipelines = [lr, lasso, dt, rf, xgb, gbr,
                eln, br, cat, lgbm] # 

    # Dictionary of pipelines and model types for ease of reference
    ml_dict = {0: "LinearRegression", 1: "Lasso", 2: "DecisionTree", 3: "RandomForest", 4: "XGBRegressor", 5: "GradientBoostingRegressor",
                    6: "Elastic Net", 7:"BayesianRidge", 8: "CatBoostRegressor", 9: "LGBMRegressor"}
        #, 

    models_scores_df = pd.DataFrame(columns=["model", "Mean CV", "MAE",
                                            "MAPE", "MSE", "RMSE", "R_Squared", "Adjusted_R_Squared"])



    # Fit the pipelines and display the scores with Cross validation
    for i, pipe in enumerate(pipelines):
        # getting the name of our model
        model_name = ml_dict[i]
        print(model_name)
        
        # fitting our data
        pipe.fit(X_train, y_train)
        
        evaluate_model(models_scores_df, i, model_name, pipe, X_train, y_train, X_test, y_test)


    # selecting top 3 score based on metric
    filtered_models_scores_df =  models_scores_df.sort_values(metric).iloc[:3, :]

    results_df = pd.DataFrame({"actual": y_test})
    selected_compounds = smiles_df.iloc[results_df.index]
    results_df['Compound'] = selected_compounds['Compound']
    results_df['SMILES'] = selected_compounds['SMILES']

    results_df = results_df.reset_index(drop=True)
    li = []

    for i in filtered_models_scores_df.index:
        predictions = pipelines[i].predict(X_test)
        
        predictions_df = pd.DataFrame({"predictions_{}".format(ml_dict[i]): predictions})
        li.append(predictions_df)
        plot_feature_importance(pipelines[i],  X.columns, ml_dict[i])

        # save the model to disk
        # filename = '../../models/{}_model.sav'.format(ml_dict[i])
        # pickle.dump(pipelines[i], open(filename, 'wb'))

    li_df = pd.concat(li, axis=1)

    return models_scores_df, pd.concat([results_df, li_df], axis=1)
# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)

In [ ]:
all_features, results_scores = train_and_evalute(X_train, X_test, y_train, y_test, metric="MAE")
all_features

In [ ]:
results_scores

<a id="ann"></a>
# 4. ANN

In [ ]:
final_result = ANN_model(X_train, X_test, y_train, y_test)
final_result

# 5. Hyperparameter Tuning

In [59]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import Adam
# Function to create the Keras model
def create_model(learning_rate=0.0001, dropout_rate=0.2):
    model = Sequential([
        Dense(256, input_shape=[X_train.shape[1]]),
        Dropout(dropout_rate),
        Dense(128),
        Dropout(dropout_rate),
        Dense(64),
        Dropout(dropout_rate),
        Dense(32),
        Dropout(dropout_rate),
        Dense(8),
        Dense(1)
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss="mean_absolute_error")
    return model

# Create KerasRegressor
model = KerasRegressor(build_fn=create_model, epochs=2500, batch_size=32, verbose=0)

# Define the grid search parameters
param_grid = {
    'learning_rate': [0.0001, 0.001, 0.01],
    'dropout_rate': [0.2, 0.3, 0.4]
}

# Use KFold cross-validation
kf = KFold(n_splits=3, shuffle=True, random_state=1)

# Create and fit the GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=kf)
grid_result = grid.fit(X_train, y_train)

# Print the results
print("Best Parameters: ", grid_result.best_params_)
print("Best MAE: ", -grid_result.best_score_)
print("Test MAE: ", -grid_result.score(X_test, y_test))

C:\Users\user\AppData\Local\Temp\ipykernel_14272\2042521271.py:26: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_model, epochs=2500, batch_size=32, verbose=0)


Best Parameters:  {'dropout_rate': 0.2, 'learning_rate': 0.001}
Best MAE:  0.5309096061913982
Test MAE:  0.36325776433187823
